# Tutorial práctico: cálculo de hash de ficheros

En este tutorial se mostrará de forma sencilla cómo generar un fichero CSV sintético y calcular su hash utilizando distintos algoritmos y tamaños de chunk.  
Esto permite entender cómo se obtiene la firma digital de un archivo y cómo los parámetros de lectura pueden influir en el resultado.

## 1. Generación de un CSV sintético

Se crea un fichero con datos ficticios de inventario para poder calcular su hash.

In [1]:
import csv

csv_file = "orders.csv"

data = [
    {"product_id": "000001", "warehouse_id": "WH_ES_01", "stock_units": 100, "reserved_units": 20},
    {"product_id": "000002", "warehouse_id": "WH_FR_01", "stock_units": 50,  "reserved_units": 5},
    {"product_id": "000003", "warehouse_id": "WH_PT_01", "stock_units": 70,  "reserved_units": 10}
]

with open(csv_file, mode="w", newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

print(f"Fichero '{csv_file}' generado correctamente.")

Fichero 'orders.csv' generado correctamente.


## 2. Función para calcular hash

Se define una función que permite calcular el hash de un fichero usando distintos algoritmos (`sha256` y `md5`) y distintos tamaños de chunk.

In [2]:
import hashlib
import binascii

def calculate_hash(file_path, algorithm='sha256', chunk_size=4096):
    """
    Calcula la firma de un fichero usando distintos algoritmos: sha256, sha512, md5, crc32.
    
    :param file_path: ruta del fichero a calcular el hash
    :param algorithm: 'sha256', 'sha512', 'md5' o 'crc32'
    :param chunk_size: tamaño de lectura por bloque en bytes
    :return: valor hash en hexadecimal
    """
    
    if algorithm == 'sha256':
        hash_func = hashlib.sha256()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(chunk_size), b''):
                hash_func.update(chunk)
        return hash_func.hexdigest()
    
    elif algorithm == 'sha512':
        hash_func = hashlib.sha512()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(chunk_size), b''):
                hash_func.update(chunk)
        return hash_func.hexdigest()
    
    elif algorithm == 'md5':
        hash_func = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(chunk_size), b''):
                hash_func.update(chunk)
        return hash_func.hexdigest()
    
    elif algorithm == 'crc32':
        crc = 0
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(chunk_size), b''):
                crc = binascii.crc32(chunk, crc)
        # Convertimos a hexadecimal de 8 dígitos
        return format(crc & 0xFFFFFFFF, '08x')
    
    else:
        raise ValueError("Algoritmo no soportado. Usa 'sha256', 'sha512', 'md5' o 'crc32'.")


## 3. Cálculo del hash con distintos algoritmos y tamaños de chunk

Se calcula el hash del fichero con SHA256 y MD5, y se experimenta con distintos valores de `chunk_size` para observar si afecta al resultado.

In [3]:
for algorithm in ['sha256', 'md5','sha512','crc32']:
    for chunk in [4096, 8192]:
        result = calculate_hash(csv_file, algorithm=algorithm, chunk_size=chunk)
        print(f"Algoritmo: {algorithm}, chunk_size: {chunk} -> hash: {result}")

Algoritmo: sha256, chunk_size: 4096 -> hash: 62495d8cd5f5d5ab7c57e5f6fcc1cdf713daefe758220d2e8dbd14d1877263db
Algoritmo: sha256, chunk_size: 8192 -> hash: 62495d8cd5f5d5ab7c57e5f6fcc1cdf713daefe758220d2e8dbd14d1877263db
Algoritmo: md5, chunk_size: 4096 -> hash: f8d2688d0b9b17681989097351c024e6
Algoritmo: md5, chunk_size: 8192 -> hash: f8d2688d0b9b17681989097351c024e6
Algoritmo: sha512, chunk_size: 4096 -> hash: f054bd979863163d678fd0916797baf1dead26857c7a26a3e855af4c5b512e5fefc72cc7540b92d010951dfa7c93e42f451c42459654c8a9aede459ab9f61c0c
Algoritmo: sha512, chunk_size: 8192 -> hash: f054bd979863163d678fd0916797baf1dead26857c7a26a3e855af4c5b512e5fefc72cc7540b92d010951dfa7c93e42f451c42459654c8a9aede459ab9f61c0c
Algoritmo: crc32, chunk_size: 4096 -> hash: 6af637c3
Algoritmo: crc32, chunk_size: 8192 -> hash: 6af637c3


### Conclusión

- El mismo fichero siempre produce la misma firma para un mismo algoritmo (`md5`, `sha256`, `sha512`, `crc32`) y tamaño de chunk.  
- Cambiar el `chunk_size` puede afectar la forma en que se leen los datos internamente, pero **no debería cambiar el hash final** si la lectura completa es correcta.  
- Los distintos algoritmos generan firmas diferentes; SHA-512 y SHA-256 son más robustos que MD5, y CRC32 es útil como verificación rápida pero no criptográficamente segura.  
- Esta función puede reutilizarse para **validar la integridad de ficheros** en pipelines de datos o procesos de control de calidad.
